In [ ]:
import gensim
import pandas as pd
import numpy as np
import requests
import re
import argparse
import sys
import os.path
import multiprocessing
!pip install docx2txt
import docx2txt

In [ ]:
def download(link, file_name):
    with open(file_name, "wb") as f:
        print("Downloading %s" % file_name)
        response = requests.get(link, stream=True)
        total_length = response.headers.get('content-length')

        if total_length is None: # no content length header
            f.write(response.content)
        else:
            dl = 0
            total_length = int(total_length)
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
                done = int(50 * dl / total_length)
                sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )
                sys.stdout.flush()

def get_id_wiki(dump_path):
    if not os.path.isfile(dump_path):
        url = 'https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2'
        download(url, dump_path)
    return gensim.corpora.WikiCorpus(dump_path, lemmatize=False, dictionary={})

In [ ]:
dump_path = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/wikipedia_corpus/idwiki-latest-pages-articles.xml.bz2'
id_wiki = get_id_wiki(dump_path)

In [ ]:
dim = 200
model_path = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/word2vec_models/idwiki_word2vec_plus_{}.model'.format(dim)
extracted_path = '/content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/wikipedia_corpus/idiwikiplus.txt'

In [ ]:
with open(extracted_path, 'w') as f:
  i_wiki = 0
  word_wiki = 0
  for text in id_wiki.get_texts():
    text = " ".join(text)
    f.write(text+'\n')
    i_wiki += 1
    word_wiki += len(text.split())

  print("Number of wikipedia texts: ",i_wiki)
  print("Number of wikipedia words: ",word_wiki)

Number of answers texts:  228
Number of answers words:  9853
Number of wikipedia texts:  394945
Number of wikipedia words:  108103162


In [ ]:
def get_w2v_model(extracted_path, model_path, dim):
  sentences = gensim.models.word2vec.LineSentence(extracted_path)
  id_w2v = gensim.models.word2vec.Word2Vec(sentences, size=dim, workers=multiprocessing.cpu_count()-1)
  id_w2v.save(model_path)
  return id_w2v

In [ ]:
print("Building Word2Vec model..")
model = get_w2v_model(extracted_path, model_path, dim)
print("Save model: ", model_path)

Building Word2Vec model..


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Save model:  /content/drive/My Drive/KP-SEMINAR-SKRIPSI/SKRIPSI/Program/Dataset/word2vec_models/idwiki_word2vec_plus_200.model


In [ ]:
model.wv.most_similar('komputer')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('laptop', 0.6610198020935059),
 ('komputasi', 0.6560447812080383),
 ('mikroprosesor', 0.6463227868080139),
 ('perangkat', 0.6400968432426453),
 ('mainframe', 0.6330007314682007),
 ('aplikasi', 0.629393994808197),
 ('cpu', 0.6240232586860657),
 ('mikrokomputer', 0.6225507855415344),
 ('smartphone', 0.5999722480773926),
 ('server', 0.5995173454284668)]

In [ ]:
!mv /content/idiwiki.txt drive/My\ Drive/
!mv /content/idwiki-latest-pages-articles.xml.bz2 drive/My\ Drive/